# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`
    -   `conda install -c conda-forge pymysql`


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host="DBMS 서버 ip",
                                     port="port번호",
                                     user="계정명",
                                     password="비밀번호",
                                     db="연결할데이터베이스이름",
                                     charset='utf8')
    ```
    - port 번호 기본값: 3306
2. Connection을 이용해 Cursor 생성  cursor .. 입출력 기능 가진! 
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
3. Cusror를 이용해 SQL문 전송
    ```python
        cursor.execute("sql문")
    ```
4. 연결 닫기
    - cursor(closing io), connection(closing network) 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


# 예제

## 테이블 생성


In [5]:
create_sql = '''
create table customer(
id int auto_increment primary key,
name varchar(20) not null, 
email varchar (50) not null unique,
tall decimal(5,2), # 100단위까지, 소숫점 2자리까지
birthday date,
created_at datetime not null

)


'''   # SQL문 마지막에 ';'붙이지 않는다.

In [17]:
import pymysql 
try:
    conn=None
    # 1. database와 연결. 
    conn= pymysql.connect(
        host='127.0.0.1', #dbms의 ip(host): str
        port=3306,        #dbms의 port 번호: int
        user='happy_noodle', #username: str
        password='3535' , # password: str
        db='mydb'
    ) # 연결 성공하면 연결된 db와 관련 작업할 수 있는 기능을 제공하는 connection 객체를 반환
    print(type(conn))
    
    # 2. connection을 사용해서 cursor 객체 생성
    # cursor : sql 처리를 하는 기능을 제공(sql 전송하고 처리결과를 받을 때 까지를 관리)
    cursor = conn.cursor()
    print(type(cursor))
    
    # 3. SQL 문 전송 + 처리결과 받기 
    
    result = cursor.execute(create_sql) # execute라는 함수가 디비로 보내주고 받아주는 
    print('처리결과:', result) # 처리결과 0이 나오는 것은 create 할 때 그런것
finally:
# 4. 연결닫기(끊기)
    if conn:
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close() # 4-2. connection 연결 닫기

<class 'pymysql.connections.Connection'>
<class 'pymysql.cursors.Cursor'>


OperationalError: (1050, "Table 'customer' already exists")

## DML

### insert


In [31]:
sql = "insert into customer (name, email, tall,birthday,created_at)\
values('이순신', 'le2e@naver.com', 185.23, '2000-09-20',now())"  # \+엔터 하면 이거 편하게 보기 위한거니까, 한줄로 처리해줘





In [33]:
# connection, cursor 생성 -> with 문으로 작성.
# manual commit이 default. 
# DML(insert/update/delete) 처리 후 commit 실행해야한다.
# try:
with pymysql.connect(host='127.0.0.1', port=3306, user='happy_noodle',password='3535',db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql) #insert/update/delete의 경우 변경된 행수를 반환
        print('결과:',result)
        conn.commit()

결과: 1


### Parameterized Query

-   Parameterized Query
    -   SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   query문을 쉽게 작성할 수 있는 장점이 있다.


In [35]:
import pymysql 
insert_sql = "insert into customer(name,email,tall,birthday,created_at)\
values(%s, %s, %s, %s, now())"

name = input('이름입력해')
email = input('이메일은?')
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql,[name, email, 173.33, '2010-01-03'])
        conn.commit()
        print('처리행:',result)

이름입력해 크어엉
이메일은? changu@gmail.com


처리행: 1


### Parameterized Query를 이용해 여러개 행 insert

#### for문 사용


# 파이썬 - sql 데이터 타입 
  str - 문자열 (char, varchar,text,...)
  int - 정수 (tinyint, int, ...)
  float - 실수(decimal, float, double, ...)
  date - date
  time - time
  datetime - datetime, timestamp

In [37]:
from datetime import date, datetime
a = date(2000,10,20) # 년 월 일 
a.year, a.month, a.day
b = datetime(2000,10,20,12,23)
b.year, b.month, b.day, b.hour, b.minute, b.second
datetime.now() #실행시점의 일시 -> datetime(mysql: now())
date.today() # 실행시점의 날짜 -> date(mysql: curdate())

datetime.date(2024, 3, 22)

In [52]:
datas = [
    ["name1", "abc1@abc.com", 165, date(2000, 1, 12), datetime.now()],
    ["name2", "def1@abc.com", 175, date(1995, 12, 20), datetime.now()],
    ["name3", "ghi1@abc.com", 185, date(1988, 7, 21), datetime.now()],
]
insert_sql = 'insert into customer(name,email,tall,birthday,created_at) values(%s,%s,%s,%s,%s)'
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
    with conn.cursor() as cursor:
        for data in datas:
            r = cursor.execute(insert_sql,data)
            conn.commit()

IntegrityError: (1062, "Duplicate entry 'abc1@abc.com' for key 'customer.email'")

#### executemany() 사용

-   insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [45]:
datas = [
    ["이름1", "abc2@a.com", 165, date(2000, 1, 12), datetime.now()],
    ["이름2", "def2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["이름1", "ghi2@a.com", 185, date(1988, 7, 21), datetime.now()],
]

In [46]:
insert_sql = 'insert into customer(name,email,tall,birthday,created_at) values(%s,%s,%s,%s,%s)'
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql,datas)
        conn.commit()

# for문을 돌려준다고 생각하면 됨 

In [47]:
print(cnt) # 모든 sql이 적용된 행수들

3


### update/delete

-   코딩 절차는 insert 와 동일


In [93]:
del float
update_sql = "update customer set tall = %s where id = %s"
tall = float(input('변경할 키:')) # 실제 DB 컬럼타입과 파이썬 타입 매칭
cust_id = int(input('변경할 고객 ID:'))# 실제 DB 컬럼타입과 파이썬 타입 매칭
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql,[tall,cust_id])
        print('처리행수:',result)
        conn.commit()


변경할 키: 180
변경할 고객 ID: 1


처리행수: 0


In [51]:
delete_sql = "delete from customer where tall > %s"

tall = float(input('삭제기준 키:'))


with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
     with conn.cursor() as cursor:
        result = cursor.execute(delete_sql,[tall])
        print('처리행수:',result)
        conn.commit()

삭제기준 키: 170


TypeError: 'float' object is not callable

## select (DQL - Data Query Language)

-   조회결과 조회
    -   cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
-   fetch메소드
    -   **fetchall()**
        -   조회한 모든 행을을 반환
    -   **fetchmany(size=개수)**
        -   지정한 size개수 만큼 반환
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [57]:
sql = "select id, name,tall from customer" # 1개 이상의 행 조회 
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
     with conn.cursor() as cursor:
         result = cursor.execute(sql)
         print('조회행수',result)
         resultset = cursor.fetchall()

조회행수 10


In [59]:
type(resultset), len(resultset)

(tuple, 10)

In [60]:
resultset

((2, '이순신', Decimal('185.23')),
 (8, '이순신', Decimal('185.23')),
 (9, '유관순', Decimal('173.33')),
 (10, '크어엉', Decimal('173.33')),
 (11, 'name1', Decimal('180.00')),
 (12, 'name2', Decimal('175.00')),
 (13, 'name3', Decimal('185.00')),
 (23, '이름1', Decimal('165.00')),
 (24, '이름2', Decimal('175.00')),
 (25, '이름1', Decimal('185.00')))

### fetchone()


In [76]:
sql = "select * from customer where id = %s" # 1개 이상의 행 조회 
cust_id =int(input('조회할 ID:'))
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
     with conn.cursor() as cursor:
         result = cursor.execute(sql,[cust_id])
         print('조회행수',result)
         resultset = cursor.fetchone()

조회할 ID: 8


조회행수 1


In [81]:
len(resultset),type(resultset)
resultset[2]

'le2e@naver.com'

In [82]:
# 딕셔너리로 뽑고싶다! 
sql = "select * from customer where id = %s" # 1개 이상의 행 조회 
cust_id =int(input('조회할 ID:'))
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb',cursorclass=pymysql.cursors.DictCursor) as conn:
     with conn.cursor() as cursor:
         result = cursor.execute(sql,[cust_id])
         print('조회행수',result)
         resultset = cursor.fetchone()

조회할 ID: 8


조회행수 1


In [83]:
type(resultset)

dict

In [85]:
resultset['name']

'이순신'

### fetchmany()


In [66]:
sql = "select * from customer" # 1개 이상의 행 조회 
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
     with conn.cursor() as cursor:
         result = cursor.execute(sql)
         print('조회행수',result)
         resultset = cursor.fetchmany(3)

조회행수 10


In [67]:
len(resultset) # 10개 다 읽어온게 아니고 3개만 읽어온 것 

3

### cursor 는 iterable 타입

-   for문에 select 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


In [96]:
sql = "select * from customer" # 실행 시 한줄씩 제공해준다.
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
     with conn.cursor() as cursor:
         result = cursor.execute(sql)
         print('조회행수',result)
         for data in cursor:
             print(data)
             print('____')

조회행수 10
(2, '이순신', 'lee@naver.com', Decimal('185.23'), datetime.date(2000, 9, 20), datetime.datetime(2024, 3, 22, 0, 0))
____
(8, '이순신', 'le2e@naver.com', Decimal('180.00'), datetime.date(2000, 9, 20), datetime.datetime(2024, 3, 22, 15, 24, 27))
____
(9, '유관순', 'r@a.com', Decimal('199.00'), datetime.date(2010, 1, 3), datetime.datetime(2024, 3, 22, 15, 32, 9))
____
(10, '크어엉', 'changu@gmail.com', Decimal('173.33'), datetime.date(2010, 1, 3), datetime.datetime(2024, 3, 22, 15, 35, 56))
____
(11, 'name1', 'abc1@abc.com', Decimal('180.00'), datetime.date(2000, 1, 12), datetime.datetime(2024, 3, 22, 16, 7, 42))
____
(12, 'name2', 'def1@abc.com', Decimal('175.00'), datetime.date(1995, 12, 20), datetime.datetime(2024, 3, 22, 16, 7, 42))
____
(13, 'name3', 'ghi1@abc.com', Decimal('185.00'), datetime.date(1988, 7, 21), datetime.datetime(2024, 3, 22, 16, 7, 42))
____
(23, '이름1', 'abc2@a.com', Decimal('165.00'), datetime.date(2000, 1, 12), datetime.datetime(2024, 3, 22, 16, 11))
____
(24, '이름2', 

In [42]:
# 업무처리 함수(로직) # 매뉴얼 커밋 형태
with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
        insert_customer(conn, "이름100", "이메일100",192.2,'1999-10-01')
        insert_customer(conn, "이름101", "이메일101",192.2,'1999-10-01')
        insert_customer(conn, "이름102", "이메일102",192.2,'1999-10-01')
        conn.rollback()
                     

NameError: name 'insert_customer' is not defined

# TODO

다음 함수들을 구현하시오.  
각 함수들은 member 테이블과 관련해서 CRUD를 처리하는 함수들 입니다.

1. name, email, tall, birthday를 매개변수로 받아서 insert하는 함수. (id는 자동증가, created_at은 실행시점의 일시가 insert되도록 한다.)

In [38]:
%%writefile customer_db.py
# autocommit 함수 -> manual commit .. 
import pymysql

def insert_member(conn, name,email,tall,birthday):
    sql = "insert into customer(name,email,tall,birthday,created_at) values(%s,%s,%s,%s,now())"
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
        with conn.cursor() as cursor:
            return cursor.execute(sql, [name,email,tall,birthday])           

Writing customer_db.py


In [ ]:
# manual commit - sql function doesn't commit. One calling must commit/rollback.
# creating connection(connecting to DB) -> caller would create 
# sql functions get connection as parameter to create a cursor.
import pymysql

def insert_member(name,email,tall,birthday):
    sql = "insert into customer(name,email,tall,birthday,created_at) values(%s,%s,%s,%s,now())"
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(sql, [name,email,tall,birthday])
            conn.commit()
    return result

def update_customer(conn, cust_id, name, email, tall, birthday):
    sql = "update customer set name =%s, email =%s, tall = %s, birthday = % where id=%s"
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
        with conn.cursor() as cursor:
            return cursor.execute(sql,[name,email,tall,birthday, cust_id])


def delete_customer_by_id(conn,cust_id):
    sql ='delete from customer where id=%s'
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
            with conn.cursor() as cursor:
                return cursor.execute(sql,[cust_id])

def select_customers(conn): # 전체조회
    sql = "select * from customer"
    with conn.cursor() as cursor:
        cursor.execute(sql)
        return cursor.fetchall()


In [10]:
insert_member('새이름', '새이메일','180.2', '2000-10-10')

1

In [39]:
import customer_db as cdb
cdb.select_customer_by_id(12)

AttributeError: module 'customer_db' has no attribute 'select_customer_by_id'

2. id, name, email, tall, birthday를 매개변수로 받아서 id의 member의 나머지 정보를 update하는 함수. (created_at은 update하지 않는다.)

In [19]:
def update_customer(cust_id, name, email, tall, birthday):
    sql = "update customer set name =%s, email =%s, tall = %s, birthday = % where id=%s"
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(sql,[name,email,tall,birthday, cust_id])
            conn.commit()
    return result

In [21]:
update_customer('바뀐이름','바뀐이메일',200.20,'2010-11-22',27)

ValueError: unsupported format character 'w' (0x77) at index 65

3. id를 매개변수로 받아서 그 member 를 삭제하는 함수.

In [23]:
def delete_customer_by_id(cust_id):
    sql ='delete from customer where id=%s'
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
            with conn.cursor() as cursor:
                result = cursor.execute(sql,[cust_id])
                conn.commit()
    return result





In [24]:
delete_customer_by_id(27)

1

5. 이름을 매개변수로 받아서 그 이름의 member를 삭제하는 함수.

In [26]:
def delete_customer_by_name(name):
    sql ='delete from customer where name=%s'
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
            with conn.cursor() as cursor:
                result = cursor.execute(sql,[name])
                conn.commit()
    return result


In [27]:
delete_customer_by_name('이름1')

2

7. id를 매개변수로 받아서 그 id의 회원 정보를 조회하여 반환하는 함수.

In [29]:
def select_customer_by_id(cust_id):
    sql = 'select * from customer where id = %s' # pk 조건으로 조회: 결과행 - 1행 => fetchone()
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
            with conn.cursor() as cursor:
                cursor.execute(sql,[cust_id]) 
                return cursor.fetchone() #커밋 할 필요 없음


In [31]:
select_customer_by_id(12)

(12,
 'name2',
 'def1@abc.com',
 Decimal('175.00'),
 datetime.date(1995, 12, 20),
 datetime.datetime(2024, 3, 22, 16, 7, 42))

8. name을 매개변수로 받아서 그 이름이 들어간 회원의 정보를 조회하여 반환하는 함수.

In [32]:
def select_customer_by_name(name):
    # 조회조건이 unique 하지 않은 컬럼: 결과행 : 0 ~N행 => fetchall()
    sql = "select * from customer where name = %s"
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
            with conn.cursor() as cursor:
                cursor.execute(sql,[name]) 
                return cursor.fetchall() 

In [33]:
select_customer_by_name('name2')

((12,
  'name2',
  'def1@abc.com',
  Decimal('175.00'),
  datetime.date(1995, 12, 20),
  datetime.datetime(2024, 3, 22, 16, 7, 42)),)

9. birthday를 매개변수로 받아서 그 생일의 회원의 정보를 조회하여 반환하는 함수.

In [ ]:
# 똑같으니 넘어감. 나는 풀어보기

10. tall 값을 두개 받아서 그 범위의 tall인 회원들의 정보를 조회하여 반환하는 함수.

-   insert, update, delete 는 적용된 행의 개수를 반환한다.
-   select 처리 함수는 조회결과를 반환한다.

In [36]:
def select_customer_by_tall_range(start_tall, stop_tall):
    sql = 'select * from customer where tall bewtween %s and %s'# like도 쓸 수 있음
    with pymysql.connect(host ='127.0.0.1', port=3306, user='happy_noodle',password ='3535', db='mydb') as conn:
            with conn.cursor() as cursor:
                cursor.execute(sql,[start_tall, stop_tall]) 
                return cursor.fetchall()
    

In [37]:
select_customer_by_tall_range(150,200)

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'bewtween 150 and 200' at line 1")

# TODO

-   hr database의 emp 테이블의 데이터를 다루는 함수들을 구현

1. 직원 정보들(emp의 컬럼들)을 매개변수로 받아 insert 하는 함수

2. 수정할 직원 정보들을 매개변수로 받아 받은 emp_id의 직원의 나머지 컬럼값들을 수정하는 함수

3. emp_id를 매개변수로 받아 그 직원을 삭제하는 함수

4. job을 매개변수로 받아 그 업무를 하는 직원들을 삭제하는 함수

5. 전체 직원정보를 모두 조회해서 반환하는 함수

6. emp_id를 매개변수로 받아 그 직원 정보를 조회해서 반환하는 함수

7. dept_name을 매개변수로 받아서 그 부서에 소속된 직원들의 정보를 조회해서 반환하는 함수

8. salary 범위를 매개변수로 받아 그 범위의 salary를 받는 직원들의 정보를 조회해서 반환하는 함수

9. emp_id를 매개변수로 받아서 그 직원의 커미션이 얼마인지를(salary \* comm_pct) 조회해 반환하는 함수

10. dept_name을 매개변수로 받아서 부서별 급여 통계정보(합계, 최대, 최소, 평균, 표준편차)를 반환하는 함수
